In [ ]:
import requests
import time
import pandas as pd
import re
from datetime import date, timedelta
import json
import io

In [ ]:
!gdown 'https://drive.google.com/uc?id=***********************'

In [ ]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('/content/holodilnik-bq-376311-50cc6f8cfc32.json')

In [ ]:
today = date.today().strftime('%Y-%m-%d')
load_date = (date.today()-timedelta(days=2)).strftime('%Y-%m-%d')

In [ ]:
CHANNEL = '*********-api-****zU'
USER_KEY = '1277b56*****************0'
EVENT = 'install'
url = f'https://get.fraudscore.ai/{CHANNEL}/{EVENT}/groups'
params = dict(key = USER_KEY)
#params['group'] = ['date','offer_name']
#params['field'] = ['datetime','any_offer_name','any_affiliate_name'] #,'id_2'
params['field'] = ['datetime','affiliate_name', 'offer_name','id_2', 'score_str']
params['sort'] = [
	'datetime_asc']
params['filter'] = json.dumps([
	"AND",
	["date_gt", load_date],
	["date_lt", today]
  ])
params['page_size'] = 1000000
r = requests.get(url, params, stream = True)


In [ ]:
datetime1 = []
offer_name = []
affiliate_name = []
id_2 = []
score_str = []

for line in r.iter_lines():
 data = json.loads(line)
 datetime1 += [data['datetime']]
 offer_name += [data['offer_name']]
 affiliate_name += [data['affiliate_name']]
 id_2 += [data['id_2']]
 score_str += [data['score_str']]
  
  
d = {'datetime1': datetime1, 'offer_name': offer_name, 'affiliate_name': affiliate_name, 'id_2': id_2, 'score': score_str}
df_fraud_score = pd.DataFrame(data=d)

In [ ]:
df_fraud_score_high = df_fraud_score.query("score == 'high'")

In [ ]:
df_fraud_score_high.to_gbq('holodilnik-bq-376311.appmetrica_dashboard.high_fraudscore_appmetrica_device_id', 
          project_id='holodilnik-bq-376311',  
          if_exists='append',
          credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 7612.17it/s]


In [ ]:
sql = """SELECT * FROM `holodilnik-bq-376311.appmetrica_dashboard.high_fraudscore_appmetrica_device_id`"""
all_data = pd.read_gbq(sql, credentials=credentials, dialect='standard')
all_data = all_data.drop_duplicates()

In [ ]:
all_data.to_gbq('holodilnik-bq-376311.appmetrica_dashboard.high_fraudscore_appmetrica_device_id', 
           project_id='holodilnik-bq-376311',  
           if_exists='replace',
           credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 5533.38it/s]
